In [51]:
import sqlalchemy 
print(sqlalchemy.__version__)

2.0.36


Establishing Connectivity - the Engine¶

In [6]:
from sqlalchemy import create_engine
engine=create_engine("sqlite+pysqlite:///:memory:", echo=True)

Getting Connection

In [53]:
from sqlalchemy import text

with engine.connect() as conn:
    result=conn.execute(text("select 'hellow world'"))
    print(result.all())

2024-10-23 21:39:14,275 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,276 INFO sqlalchemy.engine.Engine select 'hellow world'
2024-10-23 21:39:14,278 INFO sqlalchemy.engine.Engine [generated in 0.00196s] ()
[('hellow world',)]
2024-10-23 21:39:14,279 INFO sqlalchemy.engine.Engine ROLLBACK


Committing Changes

In [54]:
with engine.connect() as conn:
    conn.execute(text("create table tabb(x int, y int)"))
    conn.execute(text("insert into tabb (x,y) values (:x,:y)"),
    [{"x":1,"y":1},{"x":2,"y":4}],
    )
    conn.commit()

2024-10-23 21:39:14,281 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,281 INFO sqlalchemy.engine.Engine create table tabb(x int, y int)
2024-10-23 21:39:14,281 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ()
2024-10-23 21:39:14,292 INFO sqlalchemy.engine.Engine insert into tabb (x,y) values (?,?)
2024-10-23 21:39:14,293 INFO sqlalchemy.engine.Engine [generated in 0.00131s] [(1, 1), (2, 4)]
2024-10-23 21:39:14,294 INFO sqlalchemy.engine.Engine COMMIT


by using connect() of method we need to commit at the end but using begin() transction will be auto commited and if  any error raises it will rollback

As per doc prefered method is begin

In [55]:
with engine.begin() as conn:
    conn.execute(text("create table some_tabb(x int, y int)"))
    conn.execute(text("insert into some_tabb (x,y) values (:x,:y)"),
    [{"x":1,"y":1},{"x":2,"y":4}],
    )

2024-10-23 21:39:14,298 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,305 INFO sqlalchemy.engine.Engine create table some_tabb(x int, y int)
2024-10-23 21:39:14,305 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ()
2024-10-23 21:39:14,305 INFO sqlalchemy.engine.Engine insert into some_tabb (x,y) values (?,?)
2024-10-23 21:39:14,305 INFO sqlalchemy.engine.Engine [generated in 0.00074s] [(1, 1), (2, 4)]
2024-10-23 21:39:14,308 INFO sqlalchemy.engine.Engine COMMIT


In [56]:
with engine.connect() as conn:
    result= conn.execute(text("select x,y from some_tabb"))
    for row in result:
        print(f":x:{row.x} y:{row.y}")
    #tuple type assignment
    result = conn.execute(text("select x, y from some_tabb"))
    for x, y in result:
        print(x,y)
    #integer index
    result = conn.execute(text("select x, y from some_tabb"))
    for row in result:
        x = row[0]
        print(x)
    #Attribute name
    result=conn.execute(text("select x,y from some_tabb"))
    for row in result:
        print("row.x",row.x)
    #mapping access
    result=conn.execute(text("select x,y from some_tabb"))
    for dict_row in result.mappings():
        x=dict_row["x"]
        y=dict_row["y"]
        print("mapping",x,y)
    

2024-10-23 21:39:14,316 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,316 INFO sqlalchemy.engine.Engine select x,y from some_tabb
2024-10-23 21:39:14,316 INFO sqlalchemy.engine.Engine [generated in 0.00150s] ()
:x:1 y:1
:x:2 y:4
2024-10-23 21:39:14,316 INFO sqlalchemy.engine.Engine select x, y from some_tabb
2024-10-23 21:39:14,316 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ()
1 1
2 4
2024-10-23 21:39:14,316 INFO sqlalchemy.engine.Engine select x, y from some_tabb
2024-10-23 21:39:14,325 INFO sqlalchemy.engine.Engine [cached since 0.002552s ago] ()
1
2
2024-10-23 21:39:14,325 INFO sqlalchemy.engine.Engine select x,y from some_tabb
2024-10-23 21:39:14,328 INFO sqlalchemy.engine.Engine [cached since 0.00833s ago] ()
row.x 1
row.x 2
2024-10-23 21:39:14,359 INFO sqlalchemy.engine.Engine select x,y from some_tabb
2024-10-23 21:39:14,359 INFO sqlalchemy.engine.Engine [cached since 0.04224s ago] ()
mapping 1 1
mapping 2 4
2024-10-23 21:39:14,359 INFO sqlalchemy

select with filter params

In [57]:
with engine.begin() as conn:
    result=conn.execute(text("select x, y from some_tabb where y>:y"),{"y":2})
    for row in result:
        print(row.x,row.y)

2024-10-23 21:39:14,365 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,375 INFO sqlalchemy.engine.Engine select x, y from some_tabb where y>?
2024-10-23 21:39:14,376 INFO sqlalchemy.engine.Engine [generated in 0.00074s] (2,)
2 4
2024-10-23 21:39:14,377 INFO sqlalchemy.engine.Engine COMMIT


sending multiple parameters

In [58]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_tabb (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

#The above operation is equivalent to running the given INSERT statement once for each parameter set, except that the operation will be optimized for better performance across many rows.


2024-10-23 21:39:14,383 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,383 INFO sqlalchemy.engine.Engine INSERT INTO some_tabb (x, y) VALUES (?, ?)
2024-10-23 21:39:14,392 INFO sqlalchemy.engine.Engine [generated in 0.00148s] [(11, 12), (13, 14)]
2024-10-23 21:39:14,393 INFO sqlalchemy.engine.Engine COMMIT


Executing with an ORM Session

In [59]:
from sqlalchemy.orm import Session
stmt=text("SELECT X,Y FROM some_tabb WHERE X > :x ORDER BY X,Y")
with Session(engine) as session:
    result=session.execute(stmt,{"x":1})
    #result=session.execute(stmt,{"x" : 1})
    for row in result:
        print(f"x:{row.x} y:{row.y}")


2024-10-23 21:39:14,405 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,407 INFO sqlalchemy.engine.Engine SELECT X,Y FROM some_tabb WHERE X > ? ORDER BY X,Y
2024-10-23 21:39:14,408 INFO sqlalchemy.engine.Engine [generated in 0.00078s] (1,)
x:2 y:4
x:11 y:12
x:13 y:14
2024-10-23 21:39:14,410 INFO sqlalchemy.engine.Engine ROLLBACK


In [60]:
with Session(engine) as session:
    stmt=text("update some_tabb set x=:x where y=:y")
    result=session.execute(stmt,[{"x":10,"y":4},{"x":100,"y":12}],
                           )                       
    session.commit() 

2024-10-23 21:39:14,420 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,421 INFO sqlalchemy.engine.Engine update some_tabb set x=? where y=?
2024-10-23 21:39:14,421 INFO sqlalchemy.engine.Engine [generated in 0.00041s] [(10, 4), (100, 12)]
2024-10-23 21:39:14,422 INFO sqlalchemy.engine.Engine COMMIT


Executing with an ORM Session

In [61]:
from sqlalchemy.orm import Session

stmt=text("select x,y from some_tabb where x> :x order by x,y")
with Session(engine) as session:
    result=session.execute(stmt,{"x":11})
    for x in result:
        print(f"x: {row.x} Y: {row.y}")


2024-10-23 21:39:14,425 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,425 INFO sqlalchemy.engine.Engine select x,y from some_tabb where x> ? order by x,y
2024-10-23 21:39:14,425 INFO sqlalchemy.engine.Engine [generated in 0.00079s] (11,)
x: 13 Y: 14
x: 13 Y: 14
2024-10-23 21:39:14,425 INFO sqlalchemy.engine.Engine ROLLBACK


In [62]:
from sqlalchemy.orm import Session
stmt=text("update some_tabb set x=:x where y=:y")
with Session(engine) as session:
    result=session.execute(stmt,[{"x":11,"y":14},{"x":12,"y":13}])
    print(result)


2024-10-23 21:39:14,441 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,441 INFO sqlalchemy.engine.Engine update some_tabb set x=? where y=?
2024-10-23 21:39:14,441 INFO sqlalchemy.engine.Engine [cached since 0.02554s ago] [(11, 14), (12, 13)]
2024-10-23 21:39:14,441 INFO sqlalchemy.engine.Engine ROLLBACK


Working with Database Metadata
Setting up MetaData with Table objects¶

In [63]:
from sqlalchemy import MetaData
meta_data_obj= MetaData()


In [64]:
from sqlalchemy import Table,Column,Integer,String
user_table=Table("user_account",meta_data_obj,
                 Column("id",Integer,primary_key=True),
                 Column("name",String(30)),
                 Column("fullname",String),
                 )

When do I make a MetaData object in my program?

Having a single MetaData object for an entire application is the most common case, represented as a module-level variable in a single place in an application, often in a “models” or “dbschema” type of package. It is also very common that the MetaData is accessed via an ORM-centric registry or Declarative Base base class, so that this same MetaData is shared among ORM- and Core-declared Table objects.

There can be multiple MetaData collections as well; Table objects can refer to Table objects in other collections without restrictions. However, for groups of Table objects that are related to each other, it is in practice much more straightforward to have them set up within a single MetaData collection, both from the perspective of declaring them, as well as from the perspective of DDL (i.e. CREATE and DROP) statements being emitted in the correct order.

In [65]:
print(user_table.c.name)

user_account.name


In [66]:
print(user_table.c.keys())

['id', 'name', 'fullname']


Declaring Simple Constraints

In [67]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [68]:
from sqlalchemy import ForeignKey,Column,Integer,String
address_table=Table("address",meta_data_obj,Column("id",Integer,primary_key=True),Column("user_id",ForeignKey("user_account.id"),nullable=False),Column("email_address",String,nullable=False),)


In [69]:
print(address_table.c.keys())

['id', 'user_id', 'email_address']


In [70]:
meta_data_obj.create_all(engine)

2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine [no key 0.00124s] ()
2024-10-23 21:39:14,530 INFO sqlalchemy.engine.Engine 
C

Using ORM Declarative Forms to Define Table Metadata

Establishing a Declarative Base

In [71]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

Base.metadata
Base.registry

Declaring Mapped Classes

In [72]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [73]:
Base.metadata.create_all(engine)

2024-10-23 21:39:14,576 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,576 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-10-23 21:39:14,576 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-23 21:39:14,576 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-10-23 21:39:14,576 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-23 21:39:14,576 INFO sqlalchemy.engine.Engine COMMIT


In [74]:
class Newusers(Base):
    __tablename__ = "new_user_account"

    id = mapped_column(Integer, primary_key=True)
    name = mapped_column(String(30), nullable=False)
    fullname = mapped_column(String)

    addresses = relationship("Address", back_populates="user")

    # ... definition continues

In [75]:
from sqlalchemy import insert
stmt=insert(User).values(name="spoogebob",fullname="pranai reddy")


In [76]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [77]:
c=stmt.compile()
c.params

{'name': 'spoogebob', 'fullname': 'pranai reddy'}

In [78]:
with engine.connect() as con:
    result=con.execute(stmt)
    con.commit()


2024-10-23 21:39:14,662 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,663 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-10-23 21:39:14,664 INFO sqlalchemy.engine.Engine [generated in 0.00108s] ('spoogebob', 'pranai reddy')
2024-10-23 21:39:14,664 INFO sqlalchemy.engine.Engine COMMIT


In [79]:
result.inserted_primary_key

(1,)

In [80]:
print(insert(user_table))

INSERT INTO user_account (id, name, fullname) VALUES (:id, :name, :fullname)


In [81]:
with Session(engine) as ses:
    ses.execute(insert(user_table),[{"name":"test","fullname":"sandy_checks"},{"name":"test2","fullname":"snadychecks2"}])
    ses.commit()

2024-10-23 21:39:14,700 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,702 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-10-23 21:39:14,702 INFO sqlalchemy.engine.Engine [generated in 0.00121s] [('test', 'sandy_checks'), ('test2', 'snadychecks2')]
2024-10-23 21:39:14,702 INFO sqlalchemy.engine.Engine COMMIT


In [82]:
from sqlalchemy import select, bindparam
scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {
                "username": "spongebob",
                "email_address": "spongebob@sqlalchemy.org",
            },
            {"username": "test", "email_address": "sandy@sqlalchemy.org"},
            {"username": "test2", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2024-10-23 21:39:14,716 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:14,716 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2024-10-23 21:39:14,716 INFO sqlalchemy.engine.Engine [generated in 0.00192s] [('spongebob', 'spongebob@sqlalchemy.org'), ('test', 'sandy@sqlalchemy.org'), ('test2', 'sandy@squirrelpower.org')]
2024-10-23 21:39:14,716 INFO sqlalchemy.engine.Engine ROLLBACK


IntegrityError: (sqlite3.IntegrityError) NOT NULL constraint failed: address.user_id
[SQL: INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)]
[parameters: [('spongebob', 'spongebob@sqlalchemy.org'), ('test', 'sandy@sqlalchemy.org'), ('test2', 'sandy@squirrelpower.org')]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

The select() SQL Expression Construct

In [83]:
from sqlalchemy import select
smt=select(user_table).where(user_table.c.name=="spongebob")
print(smt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [84]:
with engine.connect() as conn:
    for row in conn.execute(smt):
        print(row)

2024-10-23 21:39:35,810 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 21:39:35,820 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-10-23 21:39:35,820 INFO sqlalchemy.engine.Engine [generated in 0.00132s] ('spongebob',)
2024-10-23 21:39:35,820 INFO sqlalchemy.engine.Engine ROLLBACK


SQLALchemy ORM

In [1]:
#Declare models
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [7]:

Base.metadata.create_all(engine)

2024-10-24 10:42:00,104 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-24 10:42:00,105 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-10-24 10:42:00,106 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-24 10:42:00,107 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-10-24 10:42:00,108 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-24 10:42:00,110 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-10-24 10:42:00,111 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-24 10:42:00,112 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-10-24 10:42:00,112 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-24 10:42:00,114 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-10-24 10:42:00,115 INFO sqlalchemy.engine.Engine [no key 0.00097s] ()
2024-10-24 10:42:00,118 INFO sqlalchemy.engine.

Create Objects and Persist

In [8]:
from sqlalchemy.orm import Session

with Session(engine) as ses:
    spongebob=User(name='spongebob',fullname="spongebob spongebob",addresses=[Address(email_address="spongbob@sqlalchemy.org")],
                   )
    pranay=User(name='pranay',fullname="pranai reddy",addresses=[Address(email_address="pranai_red@007@gmail.com"),Address(email_address="tharun@gmail.com")])
    patrick=User(name="pratrick",fullname="patrick rathod")

    ses.add_all([spongebob,pranay,patrick])
    ses.commit()

2024-10-24 11:08:07,953 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-24 11:08:07,956 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-10-24 11:08:07,957 INFO sqlalchemy.engine.Engine [generated in 0.00017s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('spongebob', 'spongebob spongebob')
2024-10-24 11:08:07,958 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-10-24 11:08:07,959 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('pranay', 'pranai reddy')
2024-10-24 11:08:07,960 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-10-24 11:08:07,961 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('pratrick', 'patrick rathod')
2024-10-24 11:08:07,964 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?) RETU

Simple SELECT¶

In [25]:
from sqlalchemy import select 

session=Session(engine)
stmt=select(User).where(User.name.in_(["spongebob","pranay"]))
for user in session.execute(stmt):
    print(user)


2024-10-24 11:21:07,783 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-24 11:21:07,784 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name IN (?, ?)
2024-10-24 11:21:07,785 INFO sqlalchemy.engine.Engine [cached since 366.9s ago] ('spongebob', 'pranay')
(User(id=1, name='spongebob', fullname='spongebob spongebob'),)
(User(id=2, name='pranay', fullname='pranai reddy'),)


select with join

In [30]:
stmt=(select(Address).join(Address.user).where(User.name =="pranay").where(Address.email_address=="pranai_red@007@gmail.com"))
pranay_address= session.scalars(stmt).one()
print(pranay_address)

2024-10-24 11:31:59,515 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = ? AND address.email_address = ?
2024-10-24 11:31:59,516 INFO sqlalchemy.engine.Engine [cached since 162.9s ago] ('pranay', 'pranai_red@007@gmail.com')
Address(id=2, email_address='pranai_red@007@gmail.com')


make changes

In [ ]:
stmt = select(User).where(User.name == "patrick")
patrick = session.scalars(stmt).one()
patrick.addresses.append(Address(email_address="patrickstar@sqlalchemy.org"))
pranay_address.email_address = "sandy_cheeks@sqlalchemy.org"

session.commit()

some deletes

In [32]:
sandy = session.get(User, 2)
sandy.addresses.remove(pranay_address)
session.flush()



2024-10-24 11:41:22,685 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2024-10-24 11:41:22,687 INFO sqlalchemy.engine.Engine [generated in 0.00168s] (2,)


In [33]:
session.delete(patrick)

2024-10-24 11:42:34,667 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2024-10-24 11:42:34,668 INFO sqlalchemy.engine.Engine [generated in 0.00138s] (3,)
2024-10-24 11:42:34,669 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2024-10-24 11:42:34,671 INFO sqlalchemy.engine.Engine [cached since 182.5s ago] (3,)


In [34]:
session.commit()

2024-10-24 11:43:06,502 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2024-10-24 11:43:06,504 INFO sqlalchemy.engine.Engine [generated in 0.00140s] (3,)
2024-10-24 11:43:06,504 INFO sqlalchemy.engine.Engine COMMIT
